In [8]:
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from preprocessing.input_data_index_embedding import *

from preprocessing.preprocessing_code_190418 import title_catcher, preprocess, date_process
import pickle as pkl
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score
from sklearn.metrics import classification_report

from collections import Counter
from konlpy.tag import Komoran
from keras.preprocessing.sequence import pad_sequences


In [9]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Input, Embedding, Dense, LSTM, Bidirectional, Dropout, Concatenate, Flatten, Conv1D, Conv2D, GlobalMaxPooling1D, TimeDistributed, SpatialDropout1D, GRU, multiply, Lambda, Reshape, CuDNNGRU, CuDNNLSTM, Permute, RepeatVector, Multiply
from keras.layers import MaxPool1D
from keras.models import Model, Sequential
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.engine.topology import Layer
from keras.preprocessing.sequence import pad_sequences
from keras_self_attention import SeqSelfAttention
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import seaborn as sn

In [10]:
import torch

from preprocessing.pytorch_pretrained_bert.tokenization_morp import BertTokenizer
from preprocessing.pytorch_pretrained_bert import BertModel, BertForMaskedLM

from matplotlib import pyplot as plt
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from eunjeon import Mecab
mecab = Mecab()

# Data Import

In [11]:
data_dir = './data/인수계약서v01.xlsx'

In [12]:
origin_data = load_dataset(data_dir)
origin_data = join_date(origin_data)

origin_data.head(3)

version v01


,text,label,par_label,id,par_id,split_id
0,제1조 (정의),-,정의,22,2,22_정의
1,본 계약에서 사용하는 용어는 다음과 같은 뜻을 가진다.,PR-02-01,정의,22,2,22_정의
2,"1. ""인수단""은 ""대표주관회사"" 및 ""인수회사""를 말한다.",PR-02-02,정의,22,2,22_정의


In [5]:
contract_names = np.unique(origin_data['id'].values)
len(contract_names)

200

# Train/Valid/Test Split

In [6]:
train, test = train_test_split(contract_names, test_size = 0.3, random_state=0)
valid, test = train_test_split(test, test_size = 0.5, random_state=0)

print(len(train), len(valid), len(test))

print('Train:',train)
print('Valid:',valid)
print('Test:',test)

140 30 30
Train: [153 118 213  41 175 114  76 185  73 108 161 112 159 123 166 111 131  36
  49 163 219  68 160 227 130  84  24  81 158 229  65  32 226  95 228 200
 197 148 115 134 180 223  72  22 116 132 117  86 189  63  91  71  70 107
  35 183  45 218 157  42  37 100 126  74 122  98  25 138 186 230  28  90
 106 143 177 193 178 113 231  33 141 124  57  79  87  23 142 184  64 127
 154 195  39  60 155  75 179 150  56  50 136 173  53 188 149 198  54 164
 191 169  51 121 104 101 137 170 225  94  99  47 187 103 220 196 222  61
  80 162 110  92 109  58  43  31 125  89 224 139  69 194]
Valid: [ 55 146 215 216  34 176 145  26 152  85 140 182 105 192 212  30  82  27
 181 151 167  96 174 128  46 120  40 119  97 190]
Test: [211 221  78 144 147 129  44  62 156  67 199  83  38  88 133 135 165 168
 102 172  93  66  48 171  59  52 217  29 214  77]


In [9]:
x_all = []
y_all = []

x_train = []
y_train = []
x_valid = []
y_valid = []
x_test = []
y_test = []

for name in contract_names:
    temp = origin_data[origin_data['id']==name]
    
    temp_contract = []
    temp_answer = []
    
    for c, l in zip(temp['text'].values, temp['label'].values):
        temp_contract.append(c)
        temp_answer.append(l)
        
    if name in train:
        x_train.append(temp_contract)
        y_train.append(temp_answer)
    elif name in valid:
        x_valid.append(temp_contract)
        y_valid.append(temp_answer)
    elif name in test:
        x_test.append(temp_contract)
        y_test.append(temp_answer)  
        
    x_all.append(temp_contract)
    y_all.append(temp_answer)

        
print(len(x_train), len(train)) 
print(len(x_valid), len(valid)) 
print(len(x_test), len(test), end='\n\n') 
print(len(y_train), len(train))
print(len(y_valid), len(valid))
print(len(y_test), len(test))

140 140
30 30
30 30

140 140
30 30
30 30


In [8]:
valid_class = np.unique(origin_data['label'].values)

In [9]:
class_size = len(valid_class)
print(class_size)

label2num = {word: i for i, word in enumerate(valid_class)}

def labels2vecs(labels):
    output = np.zeros(class_size)
    for label in labels:
        if label in label2num.keys():
            output += np.eye(class_size)[label2num[label]]
    return output.tolist()
    
num2label = {i: word for i, word in enumerate(valid_class)}

def vecs2labels(vecs):
    output = []
    for i, vec in enumerate(vecs):
        if vec == 1:
            output.append(num2label[i])
    return output

def text_preprocess(text):
    text = preprocess(text)
    text = title_process(text)
    text = time_process(text)
    text = date_process(text)
    text = phone_process(text)
    text = re.sub('[^가-힣".,()~%_ ]+', '', text)
        
    return text

250


In [11]:
valid_class = np.unique(origin_data['label'].values)
class_size = len(valid_class)
max_row = 400

In [12]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(valid_class.reshape(-1,1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

# Sentence Embedding

In [14]:
def do_lang(text):
    mecab_pos = mecab.pos(text)
    return_result = ''
    for i in mecab_pos:
        temp_text = i[0] + '/' + i[1] + ' '
        return_result = return_result + temp_text

    return return_result

In [15]:
def convert_examples_to_features(text_a, text_b, tokenizer, max_seq_length) :
    tokens_a = do_lang(text_a)
#     print(tokens_a)
    tokens_a = tokenizer.tokenize(tokens_a)
#     print(tokens_a)
    tokens_b = None
    if text_b :
        tokens_b = do_lang(text_b)
        tokens_b = tokenizer.tokenize(tokens_b)
        
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length-3)
    else :
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]
            
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)
    
    if tokens_b:
        tokens += tokens_b + ["[SEP]"]
        segment_ids += [1] * (len(tokens_b) + 1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    
    return input_ids, input_mask, segment_ids

In [16]:
def sentence_embedding(input_id, segment_id):
    
    tokens_tensor = torch.tensor([input_id], dtype=torch.long, device=device)
    segment_tensor = torch.tensor([segment_id], dtype=torch.long, device=device)
    
    
    tokens_tensor = tokens_tensor.to(device)
    segment_tensor = segment_tensor.to(device)
    
    with torch.no_grad():
        encoded_layers, _, sequence_output = model(tokens_tensor, segment_tensor)

    sentence_embedding = encoded_layers[-1][0][0]
    
    return sentence_embedding

In [17]:
def text_to_bert(text, unit, max_seq_length):
    if len(text)==2 :
        text_a=text[0]
        text_b=text[1]
    elif len(text)==1 :
        text_a=text[0]
        text_b=False
    else :
        print('check text!')
        
    input_ids, input_mask, segment_ids = convert_examples_to_features(text_a = text_a, text_b=text_b, tokenizer=tokenizer, max_seq_length=max_seq_length)

    return sentence_embedding(input_ids, segment_ids).cpu().numpy()


In [19]:
device = torch.device('cuda')
tokenizer = BertTokenizer.from_pretrained('./preprocessing/pytorch_pretrained_bert/vocab.korean_morp_mecab.list', do_lower_case=False)
model = BertModel.from_pretrained('./input','insu_bert.bin')
model.eval()
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30349, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [26]:
max_seq_length=380

In [46]:
bert_train_par = [[text_to_bert(text=[text_preprocess(j)], unit='sentence', max_seq_length=max_seq_length) for j in i] for i in x_train]
bert_valid_par = [[text_to_bert(text=[text_preprocess(j)], unit='sentence', max_seq_length=max_seq_length) for j in i] for i in x_valid]
bert_test_par = [[text_to_bert(text=[text_preprocess(j)], unit='sentence', max_seq_length=max_seq_length) for j in i] for i in x_test]

In [199]:
bert_train_par_ = pad_sequences(bert_train_par, maxlen=max_row, padding='post', dtype='float')
bert_valid_par_ = pad_sequences(bert_valid_par, maxlen=max_row, padding='post', dtype='float')
bert_test_par_ = pad_sequences(bert_test_par, maxlen=max_row, padding='post', dtype='float')

In [202]:
print('bert_train_par_', bert_train_par_.shape)
print('bert_valid_par_', bert_valid_par_.shape)
print('bert_test_par_', bert_test_par_.shape)

bert_train_par_ (140, 400, 768)
bert_valid_par_ (30, 400, 768)
bert_test_par_ (30, 400, 768)


# Model

In [32]:
# Hyper Parameter

n_words = 1792 + 2
embed_size = 200

batch_size = 8
learning_rate = 0.0001
epochs = 500

sentence_wise_lstm_size = 100

dense_dropout = 0.5
l2_reg = regularizers.l2(0.0001)

dense_size = 128
attention_dim = 250
rnn_dim = 256

In [203]:
def paragraph_lstm():
    K.clear_session()
    np.random.seed(9311)
    
    sentence_input = Input(shape=(max_row, 768), name='sentence_input')
    lstm = Bidirectional(CuDNNLSTM(128, return_sequences=True), name='paragraph_lstm')(sentence_input)
    bn = BatchNormalization()(lstm)

    attention_layer = SeqSelfAttention(units=200, 
                                       attention_activation='sigmoid', 
                                       attention_type=SeqSelfAttention.ATTENTION_TYPE_ADD,
                                       attention_width=60,
                                       kernel_regularizer=keras.regularizers.l2(1e-4),
                                       bias_regularizer=keras.regularizers.l1(1e-4),
                                       attention_regularizer_weight=1e-4,
                                       return_attention=True)(lstm)
#     do = Dropout(0.1)(attention_layer[0])
    bn = BatchNormalization()(attention_layer[0])
    output = Dense(class_size, activation='sigmoid')(bn)
    
    model = Model(inputs=sentence_input, outputs=output)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.0001),
                  metrics=['accuracy'])
    
    attention_extractor = Model(inputs=sentence_input, outputs=attention_layer[1])
    context_extractor = Model(inputs=sentence_input, outputs=attention_layer[0])
    return model, attention_extractor, context_extractor

In [204]:
model_path = './output/wise_weight/insu_bert/{epoch:02d}-{val_loss:.4f}.hdf5'

checkpointer = ModelCheckpoint(filepath=model_path,
                               monitor='val_loss',
                               verbose=True,
                               save_best_only=True)

model,extractor,context = paragraph_lstm()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sentence_input (InputLayer)  (None, 400, 768)          0         
_________________________________________________________________
paragraph_lstm (Bidirectiona (None, 400, 512)          2101248   
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 512)          2048      
_________________________________________________________________
dense_1 (Dense)              (None, 400, 250)          128250    
Total params: 2,231,546
Trainable params: 2,230,522
Non-trainable params: 1,024
_________________________________________________________________


In [205]:
hist = model.fit(bert_train_par_,y_train_,
                batch_size=4,
                epochs=500,
                verbose=2,
                validation_data=(bert_valid_par_, y_valid_),
                callbacks=[checkpointer])

Train on 140 samples, validate on 30 samples
Epoch 1/500
 - 3s - loss: 2.4781 - acc: 0.3456 - val_loss: 1.3478 - val_acc: 0.5865

Epoch 00001: saving model to ./weight/insu_bert_par/01-1.3478.hdf5
Epoch 2/500
 - 2s - loss: 0.8040 - acc: 0.6914 - val_loss: 0.4488 - val_acc: 0.7368

Epoch 00002: saving model to ./weight/insu_bert_par/02-0.4488.hdf5
Epoch 3/500
 - 2s - loss: 0.3193 - acc: 0.7426 - val_loss: 0.2285 - val_acc: 0.7559

Epoch 00003: saving model to ./weight/insu_bert_par/03-0.2285.hdf5
Epoch 4/500
 - 2s - loss: 0.2112 - acc: 0.7514 - val_loss: 0.1488 - val_acc: 0.7608

Epoch 00004: saving model to ./weight/insu_bert_par/04-0.1488.hdf5
Epoch 5/500
 - 2s - loss: 0.1796 - acc: 0.7542 - val_loss: 0.1148 - val_acc: 0.7633

Epoch 00005: saving model to ./weight/insu_bert_par/05-0.1148.hdf5
Epoch 6/500
 - 2s - loss: 0.1601 - acc: 0.7551 - val_loss: 0.0924 - val_acc: 0.7633

Epoch 00006: saving model to ./weight/insu_bert_par/06-0.0924.hdf5
Epoch 7/500
 - 2s - loss: 0.1514 - acc: 0.7


Epoch 00054: saving model to ./weight/insu_bert_par/54-0.0866.hdf5
Epoch 55/500
 - 2s - loss: 0.0411 - acc: 0.7615 - val_loss: 0.0643 - val_acc: 0.7635

Epoch 00055: saving model to ./weight/insu_bert_par/55-0.0643.hdf5
Epoch 56/500
 - 2s - loss: 0.0405 - acc: 0.7615 - val_loss: 0.0618 - val_acc: 0.7641

Epoch 00056: saving model to ./weight/insu_bert_par/56-0.0618.hdf5
Epoch 57/500
 - 2s - loss: 0.0378 - acc: 0.7620 - val_loss: 0.0710 - val_acc: 0.7627

Epoch 00057: saving model to ./weight/insu_bert_par/57-0.0710.hdf5
Epoch 58/500
 - 2s - loss: 0.0382 - acc: 0.7613 - val_loss: 0.0775 - val_acc: 0.7615

Epoch 00058: saving model to ./weight/insu_bert_par/58-0.0775.hdf5
Epoch 59/500
 - 2s - loss: 0.0426 - acc: 0.7615 - val_loss: 0.0776 - val_acc: 0.7618

Epoch 00059: saving model to ./weight/insu_bert_par/59-0.0776.hdf5
Epoch 60/500
 - 2s - loss: 0.0365 - acc: 0.7629 - val_loss: 0.0734 - val_acc: 0.7633

Epoch 00060: saving model to ./weight/insu_bert_par/60-0.0734.hdf5
Epoch 61/500
 

Epoch 108/500
 - 2s - loss: 0.0144 - acc: 0.7696 - val_loss: 0.0694 - val_acc: 0.7614

Epoch 00108: saving model to ./weight/insu_bert_par/108-0.0694.hdf5
Epoch 109/500
 - 2s - loss: 0.0136 - acc: 0.7699 - val_loss: 0.0627 - val_acc: 0.7622

Epoch 00109: saving model to ./weight/insu_bert_par/109-0.0627.hdf5
Epoch 110/500
 - 2s - loss: 0.0121 - acc: 0.7700 - val_loss: 0.0670 - val_acc: 0.7647

Epoch 00110: saving model to ./weight/insu_bert_par/110-0.0670.hdf5
Epoch 111/500
 - 2s - loss: 0.0117 - acc: 0.7699 - val_loss: 0.0606 - val_acc: 0.7643

Epoch 00111: saving model to ./weight/insu_bert_par/111-0.0606.hdf5
Epoch 112/500
 - 2s - loss: 0.0115 - acc: 0.7697 - val_loss: 0.0625 - val_acc: 0.7647

Epoch 00112: saving model to ./weight/insu_bert_par/112-0.0625.hdf5
Epoch 113/500
 - 2s - loss: 0.0113 - acc: 0.7702 - val_loss: 0.0615 - val_acc: 0.7638

Epoch 00113: saving model to ./weight/insu_bert_par/113-0.0615.hdf5
Epoch 114/500
 - 2s - loss: 0.0108 - acc: 0.7705 - val_loss: 0.0582 - 


Epoch 00213: saving model to ./weight/insu_bert_par/213-0.0663.hdf5
Epoch 214/500
 - 2s - loss: 0.0064 - acc: 0.7712 - val_loss: 0.0673 - val_acc: 0.7640

Epoch 00214: saving model to ./weight/insu_bert_par/214-0.0673.hdf5
Epoch 215/500
 - 2s - loss: 0.0058 - acc: 0.7714 - val_loss: 0.0677 - val_acc: 0.7647

Epoch 00215: saving model to ./weight/insu_bert_par/215-0.0677.hdf5
Epoch 216/500
 - 2s - loss: 0.0059 - acc: 0.7716 - val_loss: 0.0668 - val_acc: 0.7639

Epoch 00216: saving model to ./weight/insu_bert_par/216-0.0668.hdf5
Epoch 217/500
 - 2s - loss: 0.0060 - acc: 0.7713 - val_loss: 0.0681 - val_acc: 0.7639

Epoch 00217: saving model to ./weight/insu_bert_par/217-0.0681.hdf5
Epoch 218/500
 - 2s - loss: 0.0060 - acc: 0.7711 - val_loss: 0.0673 - val_acc: 0.7643

Epoch 00218: saving model to ./weight/insu_bert_par/218-0.0673.hdf5
Epoch 219/500
 - 2s - loss: 0.0056 - acc: 0.7715 - val_loss: 0.0708 - val_acc: 0.7637

Epoch 00219: saving model to ./weight/insu_bert_par/219-0.0708.hdf5
E


Epoch 00266: saving model to ./weight/insu_bert_par/266-0.0614.hdf5
Epoch 267/500
 - 2s - loss: 0.0100 - acc: 0.7700 - val_loss: 0.0960 - val_acc: 0.7573

Epoch 00267: saving model to ./weight/insu_bert_par/267-0.0960.hdf5
Epoch 268/500
 - 2s - loss: 0.0103 - acc: 0.7696 - val_loss: 0.0757 - val_acc: 0.7609

Epoch 00268: saving model to ./weight/insu_bert_par/268-0.0757.hdf5
Epoch 269/500
 - 2s - loss: 0.0065 - acc: 0.7714 - val_loss: 0.0677 - val_acc: 0.7643

Epoch 00269: saving model to ./weight/insu_bert_par/269-0.0677.hdf5
Epoch 270/500
 - 2s - loss: 0.0060 - acc: 0.7716 - val_loss: 0.0726 - val_acc: 0.7619

Epoch 00270: saving model to ./weight/insu_bert_par/270-0.0726.hdf5
Epoch 271/500
 - 2s - loss: 0.0053 - acc: 0.7720 - val_loss: 0.0717 - val_acc: 0.7629

Epoch 00271: saving model to ./weight/insu_bert_par/271-0.0717.hdf5
Epoch 272/500
 - 2s - loss: 0.0053 - acc: 0.7718 - val_loss: 0.0654 - val_acc: 0.7646

Epoch 00272: saving model to ./weight/insu_bert_par/272-0.0654.hdf5
E


Epoch 00319: saving model to ./weight/insu_bert_par/319-0.0710.hdf5
Epoch 320/500
 - 2s - loss: 0.0034 - acc: 0.7724 - val_loss: 0.0801 - val_acc: 0.7626

Epoch 00320: saving model to ./weight/insu_bert_par/320-0.0801.hdf5
Epoch 321/500
 - 2s - loss: 0.0050 - acc: 0.7719 - val_loss: 0.0742 - val_acc: 0.7632

Epoch 00321: saving model to ./weight/insu_bert_par/321-0.0742.hdf5
Epoch 322/500
 - 2s - loss: 0.0043 - acc: 0.7721 - val_loss: 0.0752 - val_acc: 0.7621

Epoch 00322: saving model to ./weight/insu_bert_par/322-0.0752.hdf5
Epoch 323/500
 - 2s - loss: 0.0076 - acc: 0.7716 - val_loss: 0.2096 - val_acc: 0.7218

Epoch 00323: saving model to ./weight/insu_bert_par/323-0.2096.hdf5
Epoch 324/500
 - 2s - loss: 0.0179 - acc: 0.7683 - val_loss: 0.1059 - val_acc: 0.7533

Epoch 00324: saving model to ./weight/insu_bert_par/324-0.1059.hdf5
Epoch 325/500
 - 2s - loss: 0.0219 - acc: 0.7684 - val_loss: 0.2468 - val_acc: 0.7152

Epoch 00325: saving model to ./weight/insu_bert_par/325-0.2468.hdf5
E


Epoch 00372: saving model to ./weight/insu_bert_par/372-0.0776.hdf5
Epoch 373/500
 - 2s - loss: 0.0025 - acc: 0.7729 - val_loss: 0.0730 - val_acc: 0.7645

Epoch 00373: saving model to ./weight/insu_bert_par/373-0.0730.hdf5
Epoch 374/500
 - 2s - loss: 0.0026 - acc: 0.7729 - val_loss: 0.0749 - val_acc: 0.7638

Epoch 00374: saving model to ./weight/insu_bert_par/374-0.0749.hdf5
Epoch 375/500
 - 2s - loss: 0.0026 - acc: 0.7726 - val_loss: 0.0739 - val_acc: 0.7641

Epoch 00375: saving model to ./weight/insu_bert_par/375-0.0739.hdf5
Epoch 376/500
 - 2s - loss: 0.0032 - acc: 0.7727 - val_loss: 0.0753 - val_acc: 0.7636

Epoch 00376: saving model to ./weight/insu_bert_par/376-0.0753.hdf5
Epoch 377/500
 - 2s - loss: 0.0025 - acc: 0.7727 - val_loss: 0.0754 - val_acc: 0.7630

Epoch 00377: saving model to ./weight/insu_bert_par/377-0.0754.hdf5
Epoch 378/500
 - 2s - loss: 0.0025 - acc: 0.7727 - val_loss: 0.0764 - val_acc: 0.7624

Epoch 00378: saving model to ./weight/insu_bert_par/378-0.0764.hdf5
E


Epoch 00425: saving model to ./weight/insu_bert_par/425-0.0719.hdf5
Epoch 426/500
 - 2s - loss: 0.0032 - acc: 0.7726 - val_loss: 0.0739 - val_acc: 0.7633

Epoch 00426: saving model to ./weight/insu_bert_par/426-0.0739.hdf5
Epoch 427/500
 - 2s - loss: 0.0030 - acc: 0.7726 - val_loss: 0.0727 - val_acc: 0.7638

Epoch 00427: saving model to ./weight/insu_bert_par/427-0.0727.hdf5
Epoch 428/500
 - 2s - loss: 0.0027 - acc: 0.7727 - val_loss: 0.0714 - val_acc: 0.7643

Epoch 00428: saving model to ./weight/insu_bert_par/428-0.0714.hdf5
Epoch 429/500
 - 2s - loss: 0.0024 - acc: 0.7729 - val_loss: 0.0738 - val_acc: 0.7642

Epoch 00429: saving model to ./weight/insu_bert_par/429-0.0738.hdf5
Epoch 430/500
 - 2s - loss: 0.0028 - acc: 0.7727 - val_loss: 0.0778 - val_acc: 0.7633

Epoch 00430: saving model to ./weight/insu_bert_par/430-0.0778.hdf5
Epoch 431/500
 - 2s - loss: 0.0025 - acc: 0.7728 - val_loss: 0.0693 - val_acc: 0.7649

Epoch 00431: saving model to ./weight/insu_bert_par/431-0.0693.hdf5
E


Epoch 00478: saving model to ./weight/insu_bert_par/478-0.0753.hdf5
Epoch 479/500
 - 2s - loss: 0.0011 - acc: 0.7733 - val_loss: 0.0813 - val_acc: 0.7643

Epoch 00479: saving model to ./weight/insu_bert_par/479-0.0813.hdf5
Epoch 480/500
 - 2s - loss: 9.7355e-04 - acc: 0.7734 - val_loss: 0.0765 - val_acc: 0.7644

Epoch 00480: saving model to ./weight/insu_bert_par/480-0.0765.hdf5
Epoch 481/500
 - 2s - loss: 0.0012 - acc: 0.7733 - val_loss: 0.0817 - val_acc: 0.7641

Epoch 00481: saving model to ./weight/insu_bert_par/481-0.0817.hdf5
Epoch 482/500
 - 2s - loss: 0.0015 - acc: 0.7732 - val_loss: 0.0789 - val_acc: 0.7633

Epoch 00482: saving model to ./weight/insu_bert_par/482-0.0789.hdf5
Epoch 483/500
 - 2s - loss: 0.0014 - acc: 0.7732 - val_loss: 0.0749 - val_acc: 0.7648

Epoch 00483: saving model to ./weight/insu_bert_par/483-0.0749.hdf5
Epoch 484/500
 - 2s - loss: 0.0013 - acc: 0.7732 - val_loss: 0.0778 - val_acc: 0.7644

Epoch 00484: saving model to ./weight/insu_bert_par/484-0.0778.hd

# Evaluation

In [94]:
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [220]:
# model.load_weights('./weight/insu_bert_par/65-0.0856.hdf5')

In [221]:
def evaluate(input_data, real, model):
    
    pred_data = np.concatenate(model.predict(input_data, batch_size=1))
    real_label = np.array([np.argmax(i) for i in np.concatenate(real)])
    padding_except_idx = np.array([idx for idx, i in enumerate(real_label) if np.sum(i)!=0])
        
    real_label = real_label[padding_except_idx]
    pred_label = np.array([np.argmax(i) for i in pred_data])[padding_except_idx]
    accuracy = np.mean([pred==real for pred, real in zip(pred_label, real_label)])
    
    print('precision', precision_score(real_label, pred_label, average='macro'))
    print('recall', recall_score(real_label, pred_label, average='macro'))
    print('f1 score', f1_score(real_label, pred_label, average='macro'))
    
    return [accuracy, classification_report(real_label, pred_label, output_dict=True)]

In [222]:
# train_accuracy = print('train accuracy : {}'.format(evaluate(input_data=bert_train_par_, real=y_train_, model=model)[0]), end='\n\n')
# valid_accuracy = print('valid accuracy : {}'.format(evaluate(input_data=bert_valid_par_, real=y_valid_, model=model)[0]), end='\n\n')
test_accuracy = print('test accuracy : {}'.format(evaluate(input_data=bert_test_par_, real=y_test_, model=model)[0]), end='\n\n')

precision 0.9739492087980604
recall 0.9780361287086813
f1 score 0.9736909463776847
test accuracy : 0.984673426078755



In [148]:
report = pd.DataFrame(evaluate(bert_test_par_[0:1], y_test_[0:1], model)[1]).T

precision 0.9770833333333333
recall 0.9751328502415458
f1 score 0.9747006122607147


# Debugging

In [81]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(valid_class.reshape(-1,1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [82]:
padding_idx = np.array([idx for idx, i in enumerate(np.concatenate(y_test_)) if np.sum(i)!=0])

In [83]:
real_vectors = [i for i in np.concatenate(y_test_)]
real_labels = enc.inverse_transform(np.array(real_vectors))[padding_idx]

In [86]:
pred_data = np.concatenate(model.predict([bert_test,bert_test_par_], batch_size=1))[padding_idx]
pred_labels = enc.inverse_transform(pred_data)

In [87]:
concord = [i==j for i,j in zip(real_labels, pred_labels)]

In [89]:
output = pd.DataFrame({'text': list(np.concatenate(x_test)), 'real':list(real_labels), 'pred':list(pred_labels), 'check':concord})
output.to_excel('./output/인수계약서_분류결과.xlsx')